First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import cv2
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 3)
model = model.cuda().eval().half()

In [2]:
# model.load_state_dict(torch.load('kiri_main.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [4]:
# torch.save(model_trt.state_dict(), 'kiri_main_trt.pth')

Load the optimized model by executing the cell below

In [5]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_main.pth'))

<All keys matched successfully>

In [6]:
model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('kiri_main_trt.pth')) # well trained collision avoidance model

<All keys matched successfully>

Create the racecar class

In [7]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [8]:
!echo $USER | sudo -S systemctl restart nvargus-daemon 

[sudo] password for jetson: 

In [9]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

Create the camera class.

In [12]:
'''import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets

network_output_slider = widgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='Steering Gain', min=-1.0, max=1.0, value=-0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='Steering Bias', min=-0.5, max=0.5, value=0.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='Steering', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='Throttle', min=-1.0, max=1.0, value=0.15, step=0.01, orientation='vertical')


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([network_output_slider,
                       widgets.Label(value="X"),
                       steering_gain_slider,
                       widgets.Label(value="+"),
                       steering_bias_slider,
                       widgets.Label(value="||"), 
                       steering_value_slider], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         live_execution_widget,
         throttle_slider]
    )
)'''

'''
from utils import preprocess
import numpy as np

STEERING_GAIN = 1.2
STEERING_BIAS = 0

car.throttle = 0.6
while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = -(x * STEERING_GAIN + STEERING_BIAS)
'''

import torch.nn.functional as F
from utils import preprocess
import numpy as np
import time

#STEERING_GAIN = 0.65
#STEERING_BIAS = 0
Kp = 1.65
kd = 5
last_x= 0
car.throttle = 0

start_time = time.time()

while True:  
    current_time = time.time()
    elapsed_time = current_time - start_time
    
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    prob_blocked = float(F.softmax(model_trt_collision(image), dim=1).flatten()[1])

    x = float(output[0])
    
    if(prob_blocked > 0.1):
        car.steering = -(x * Kp + (x - last_x) * kd)
    else:        
        car.steering = 0.6
        time.sleep(0.4)
        
    
  
        
    
    
# #     print('====================================================================')
#     print("Waktu : ", elapsed_time)
#     print("Road Following : ",x)
# #     print('====================================================================')
#     print("object A: ", a)
#     print('====================================================================')
# #     print("Model B: ", b)
#     print('====================================================================')
#     print("model belok: ", prob_blocked)
#     print('====================================================================')

    

#    # 2. STOP MANUAL Mengubah car.steering sesuai waktu yang ditentukan 
#     if change_steering_time <= elapsed_time <= delay_steering_time:
#         car.steering = 1
#     else:
#         car.steering = -(x * Kp + (x - last_x) * kd)

    # Mengubah car.throttle sesuai waktu yang ditentukan
#     if change_throttle_time <= elapsed_time <= delay_throttle_time:
#         car.throttle = 0.4
#     elif change_throttle_time2 <= elapsed_time <= delay_throttle_time2:
#         car.throttle = 0.3
#     else:
#         car.throttle = 0

    # Hentikan loop jika Anda mencapai waktu tertentu (misalnya 30 detik)
#     if elapsed_time >= 12:
#         break
    
    

    # Hentikan loop jika Anda mencapai waktu tertentu (misalnya 30 detik)
#     if current_time >= 120:
#         break
    
#     current_time = time.time()
#     if current_time - start_time >= 20:
#         car.throttle = 0 
#         if throttle_change_time is None:
#             throttle_change_time = current_time  

#     if throttle_change_time is not None and current_time - throttle_change_time >= 5:
#         car.throttle = 0 

#     # Debug delay time
#     print("waktu berlangsung: ",elapsed_time)
#     print("--------------------------------------------")
#     print("             ")
#     print("waktu awal: ",current_time)
#     print("============================================")
#     print("belak belok :", car.steering)
#     print("--------------------------------------------")
#     print("kecepatan :", car.throttle)
#     print("============================================")
    


    
#     #3. Digunakan untuk belok lebih smooth

#     if (car.steering == 0):
#         car.throttle = 0.35
#     elif (car.steering == 1) :
#         car.throttle = 0.25
#     elif (car.steering == -1) :
#         car.throttle = 0.25
#     else:
#         car.throttle = 0.3
    
    
    
    
    

KeyboardInterrupt: 

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)